In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
import numpy as np

Implement a class that works the same as sklearn knn-algorithm.

**Small tip:** you can make two classes:

1) class MyNNEstimator(BaseEstimator, ClassifierMixin);

2) class MyNNClassifier(MyNNEstimator):

After that, you'll have to check your algorithm on the data below and achive max f1 measure.

f1 measure function should be implemented manually. 

__Small tip:__ use space transformations

# Data

In [3]:
data = pd.read_csv('chips.txt')
data.head()

,x,y,cls
0,0.051267,0.69956,1
1,-0.092742,0.68494,1
2,-0.213710,0.69225,1
3,-0.375000,0.50219,1
4,-0.513250,0.46564,1


In [4]:
data['cls'].value_counts()

0    60
1    58
Name: cls, dtype: int64

In [5]:
X = data.loc[:,['x', 'y']]
y = data.loc[:,'cls']
X.head()
y.head()

,x,y
0,0.051267,0.69956
1,-0.092742,0.68494
2,-0.213710,0.69225
3,-0.375000,0.50219
4,-0.513250,0.46564


0    1
1    1
2    1
3    1
4    1
Name: cls, dtype: int64

In [6]:
# Разбиваем на тестовую и обучающую выборки
np.random.seed(112358)
test_inds = np.random.choice(data.index, replace=False,
                             size = round(len(data.index) * 1 / 3))
test_inds.sort()
test_inds

array([  1,   2,   7,  12,  15,  16,  18,  23,  25,  27,  30,  32,  36,
        37,  52,  54,  57,  58,  62,  63,  68,  70,  72,  76,  78,  80,
        82,  86,  87,  89,  91,  92,  96,  99, 104, 106, 107, 111, 114])

In [7]:
X_test = data[['x', 'y']].iloc[test_inds]
X_train = data[['x', 'y']].drop(test_inds)

X_test.shape
X_train.shape

(39, 2)

(79, 2)

In [8]:
y_test = data['cls'].iloc[test_inds]
y_train = data['cls'].drop(test_inds)

y_test.size
y_train.size

39

79

# Sklearn-Knn

In [9]:
from sklearn.neighbors import KNeighborsClassifier

In [10]:
temp_cls = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

In [11]:
temp_cls.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [12]:
temp_cls.score(X_train, y_train)
temp_cls.score(X_test, y_test)

0.83544303797468356

0.79487179487179482

In [13]:
# temp_cls.predict?

In [14]:
temp_cls.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1])

In [15]:
temp_cls.predict_proba(X_test)[:10]

array([[ 0. ,  1. ],
       [ 0.2,  0.8],
       [ 0.4,  0.6],
       [ 0.4,  0.6],
       [ 0.2,  0.8],
       [ 0.2,  0.8],
       [ 0. ,  1. ],
       [ 1. ,  0. ],
       [ 0.4,  0.6],
       [ 0.6,  0.4]])

In [16]:
from sklearn.metrics import f1_score
f1_score(y_train, temp_cls.predict(X_train))
f1_score(y_test, temp_cls.predict(X_test))

0.84705882352941186

0.77777777777777779

In [17]:
# temp_cls.weights?

In [18]:
temp_cls.get_params()

temp_cls._get_param_names()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': -1,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

['algorithm',
 'leaf_size',
 'metric',
 'metric_params',
 'n_jobs',
 'n_neighbors',
 'p',
 'weights']

In [19]:
temp_cls._y
temp_cls._fit_X[:5]
temp_cls._estimator_type

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

array([[ 0.051267,  0.69956 ],
       [-0.375   ,  0.50219 ],
       [-0.51325 ,  0.46564 ],
       [-0.52477 ,  0.2098  ],
       [-0.39804 ,  0.034357]])

'classifier'

## Cross Validation

## 1. CV by hands

In [20]:
SEED = 112358
N_FOLDS = 5

accuracy_train = []
accuracy_test = []
f1_score_train = []
f1_score_test = []

In [21]:
# Разбиваем на тестовую и обучающую выборки
for i in range(N_FOLDS):
    np.random.seed(SEED + i)
    test_inds = np.random.choice(data.index, replace=False,
                                 size = round(len(data.index) * 1 / N_FOLDS))
    test_inds.sort()

    X_test = data[['x', 'y']].iloc[test_inds]
    X_train = data[['x', 'y']].drop(test_inds)

    y_test = data['cls'].iloc[test_inds]
    y_train = data['cls'].drop(test_inds)

    temp_cls = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
    temp_cls.fit(X_train, y_train)

    print('Accuracy:')
    accuracy_train.append(temp_cls.score(X_train, y_train))
    accuracy_test.append(temp_cls.score(X_test, y_test))

    print('f1_score:')
    f1_score_train.append(f1_score(y_train, temp_cls.predict(X_train)))
    f1_score_test.append(f1_score(y_test, temp_cls.predict(X_test)))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

Accuracy:
f1_score:


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

Accuracy:
f1_score:


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

Accuracy:
f1_score:


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

Accuracy:
f1_score:


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

Accuracy:
f1_score:


In [22]:
print('Accuracy of train: %s' % np.mean(accuracy_train))
print('Accuracy of test:  %s' % np.mean(accuracy_test))
print('f1_score of train: %s' % np.mean(f1_score_train))
print('f1_score of test:  %s' % np.mean(f1_score_test))

Accuracy of train: 0.83829787234
Accuracy of test:  0.733333333333
f1_score of train: 0.8467731423
f1_score of test:  0.706066461719


## 2. CV cross_val_score

In [23]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [24]:
temp_cls = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

In [25]:
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state = 112358) # shuffle=True - важно

cross_val_score(temp_cls, X, y, scoring='f1', cv=skf, n_jobs=-1)

array([ 0.7826087 ,  0.8       ,  0.57142857,  0.86956522,  0.86956522])

## 3. CV by GridSearchCV

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
temp_cls = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

In [28]:
param_grid = {
    'n_neighbors':range(5, 10),
    'metric':['cityblock', 'minkowski'],
    'p': [2, 3, 4]
}

In [29]:
neigh_cv = GridSearchCV(temp_cls, param_grid,
                        cv=skf, n_jobs=-1,
                        scoring='f1',
                        error_score='f1',
                        verbose=True, refit=True)

In [30]:
neigh_cv.fit(X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    8.5s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=112358, shuffle=True),
       error_score='f1',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_neighbors': range(5, 10), 'metric': ['cityblock', 'minkowski'], 'p': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=True)

In [31]:
neigh_cv.grid_scores_

/Users/max/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.83223, std: 0.08343, params: {'metric': 'cityblock', 'n_neighbors': 5, 'p': 2},
 mean: 0.83223, std: 0.08343, params: {'metric': 'cityblock', 'n_neighbors': 5, 'p': 3},
 mean: 0.83223, std: 0.08343, params: {'metric': 'cityblock', 'n_neighbors': 5, 'p': 4},
 mean: 0.70155, std: 0.12930, params: {'metric': 'cityblock', 'n_neighbors': 6, 'p': 2},
 mean: 0.70155, std: 0.12930, params: {'metric': 'cityblock', 'n_neighbors': 6, 'p': 3},
 mean: 0.70155, std: 0.12930, params: {'metric': 'cityblock', 'n_neighbors': 6, 'p': 4},
 mean: 0.72585, std: 0.13028, params: {'metric': 'cityblock', 'n_neighbors': 7, 'p': 2},
 mean: 0.72585, std: 0.13028, params: {'metric': 'cityblock', 'n_neighbors': 7, 'p': 3},
 mean: 0.72585, std: 0.13028, params: {'metric': 'cityblock', 'n_neighbors': 7, 'p': 4},
 mean: 0.72785, std: 0.16902, params: {'metric': 'cityblock', 'n_neighbors': 8, 'p': 2},
 mean: 0.72785, std: 0.16902, params: {'metric': 'cityblock', 'n_neighbors': 8, 'p': 3},
 mean: 0.72785, std: 

In [33]:
neigh_cv.best_estimator_
neigh_cv.best_params_
neigh_cv.best_score_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cityblock',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

{'metric': 'cityblock', 'n_neighbors': 5, 'p': 2}

0.83222795381970038

# MyNNClassifier

In [34]:
from os import getcwd
from sys import path

path.append(getcwd())
path

['',
 '/Users/max/anaconda3/lib/python36.zip',
 '/Users/max/anaconda3/lib/python3.6',
 '/Users/max/anaconda3/lib/python3.6/lib-dynload',
 '/Users/max/anaconda3/lib/python3.6/site-packages',
 '/Users/max/anaconda3/lib/python3.6/site-packages/aeosa',
 '/Users/max/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/Users/max/.ipython',
 '/Users/max/Yandex.Disk.localized/Study/ITMO/1 semestr/Python/[Done] 3_practice_01_11_2017']

In [35]:
from MyNNClassifier import MyNNClassifier

In [36]:
my_NN = MyNNClassifier(n_neighbors = 5, scaler = 'None', p = 1) #, scaler = 'MinMaxScaler'

In [37]:
my_NN._get_param_names()
my_NN.get_params()
my_NN._estimator_type

['metric', 'n_neighbors', 'p', 'scaler', 'weights']

{'metric': 'minkowski',
 'n_neighbors': 5,
 'p': 1,
 'scaler': 'None',
 'weights': 'uniform'}

'classifier'

In [38]:
my_NN.set_params(n_neighbors = 4, scaler = 'StandardScaler', p=2)

MyNNClassifier(metric='minkowski', n_neighbors=4, p=2,
        scaler='StandardScaler', weights='uniform')

In [39]:
my_NN.fit(X_train, y_train)

In [40]:
my_NN._predict(X_test)

array([ 1.  ,  0.75,  0.5 ,  0.5 ,  0.75,  0.75,  0.5 ,  0.5 ,  1.  ,
        1.  ,  1.  ,  0.75,  1.  ,  0.  ,  0.25,  0.25,  0.75,  0.  ,
        0.75,  0.5 ,  0.  ,  0.75,  0.  ,  0.25])

In [41]:
my_NN.predict(X_test)

array([ 1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,
        0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.])

In [42]:
my_NN.predict_proba(X_test)[:10]

array([[ 0.  ,  1.  ],
       [ 0.25,  0.75],
       [ 0.5 ,  0.5 ],
       [ 0.5 ,  0.5 ],
       [ 0.25,  0.75],
       [ 0.25,  0.75],
       [ 0.5 ,  0.5 ],
       [ 0.5 ,  0.5 ],
       [ 0.  ,  1.  ],
       [ 0.  ,  1.  ]])

In [43]:
my_NN.score(X_train, y_train)
my_NN.score(X_test, y_test)

0.82352941176470595

0.71999999999999986

In [44]:
my_NN.get_params()
my_NN.set_params(scaler = 'MinMaxScaler')
my_NN.get_params()

{'metric': 'minkowski',
 'n_neighbors': 4,
 'p': 2,
 'scaler': 'StandardScaler',
 'weights': 'uniform'}

MyNNClassifier(metric='minkowski', n_neighbors=4, p=2, scaler='MinMaxScaler',
        weights='uniform')

{'metric': 'minkowski',
 'n_neighbors': 4,
 'p': 2,
 'scaler': 'MinMaxScaler',
 'weights': 'uniform'}

# GridSearchCV

In [45]:
from sklearn.model_selection import GridSearchCV

In [46]:
param_grid = {
    'n_neighbors':range(5, 10),
    'scaler':['StandardScaler'],
    'metric':['cityblock', 'minkowski'],
    'p':[2, 3, 4]
}

In [47]:
my_NN_cv = GridSearchCV(my_NN, param_grid,
                        cv=skf, n_jobs=-1,
                        scoring='f1',
                        verbose=True, refit=True)

In [48]:
my_NN_cv.fit(X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   14.7s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=112358, shuffle=True),
       error_score='raise',
       estimator=MyNNClassifier(metric='minkowski', n_neighbors=4, p=2, scaler='MinMaxScaler',
        weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_neighbors': range(5, 10), 'scaler': ['StandardScaler'], 'metric': ['cityblock', 'minkowski'], 'p': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=True)

In [49]:
my_NN_cv.grid_scores_

/Users/max/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.80116, std: 0.10408, params: {'metric': 'cityblock', 'n_neighbors': 5, 'p': 2, 'scaler': 'StandardScaler'},
 mean: 0.80116, std: 0.10408, params: {'metric': 'cityblock', 'n_neighbors': 5, 'p': 3, 'scaler': 'StandardScaler'},
 mean: 0.80116, std: 0.10408, params: {'metric': 'cityblock', 'n_neighbors': 5, 'p': 4, 'scaler': 'StandardScaler'},
 mean: 0.70155, std: 0.12930, params: {'metric': 'cityblock', 'n_neighbors': 6, 'p': 2, 'scaler': 'StandardScaler'},
 mean: 0.70155, std: 0.12930, params: {'metric': 'cityblock', 'n_neighbors': 6, 'p': 3, 'scaler': 'StandardScaler'},
 mean: 0.70155, std: 0.12930, params: {'metric': 'cityblock', 'n_neighbors': 6, 'p': 4, 'scaler': 'StandardScaler'},
 mean: 0.73234, std: 0.13461, params: {'metric': 'cityblock', 'n_neighbors': 7, 'p': 2, 'scaler': 'StandardScaler'},
 mean: 0.73234, std: 0.13461, params: {'metric': 'cityblock', 'n_neighbors': 7, 'p': 3, 'scaler': 'StandardScaler'},
 mean: 0.73234, std: 0.13461, params: {'metric': 'cityblock', 'n

In [50]:
my_NN_cv.best_estimator_
my_NN_cv.best_params_
my_NN_cv.best_score_

MyNNClassifier(metric='cityblock', n_neighbors=5, p=2,
        scaler='StandardScaler', weights='uniform')

{'metric': 'cityblock', 'n_neighbors': 5, 'p': 2, 'scaler': 'StandardScaler'}

0.80115562403697993

# Draft

In [301]:
from MyNNEstimator import MyNNEstimator

import numpy as np

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.spatial.distance import cdist
from scipy.spatial.distance import braycurtis, canberra, chebyshev, cityblock, \
                                   correlation, cosine, dice, euclidean, hamming, \
                                   jaccard, kulsinski, mahalanobis, matching, minkowski, \
                                   rogerstanimoto, russellrao, seuclidean, sokalmichener, \
                                   sokalsneath, sqeuclidean, wminkowski, yule



class MyNNClassifier(MyNNEstimator):
    """
    Inherited methods:
    BaseEstimator - get_params, set_params
    ClassifierMixin - score
    ------------------------
    Classifier implementing the k-nearest neighbors vote.

    Read more in the :ref:`User Guide <classification>`.

    Parameters
    ------------------------
    n_neighbors : int, optional (default = 5)
        Number of neighbors to use by default for :meth:`kneighbors` queries.

    metric : string or callable, default 'minkowski'
        the distance metric to use for the tree.  The default metric is
        minkowski, and with p=2 is equivalent to the standard Euclidean
        metric. See the documentation of the DistanceMetric class for a
        list of available metrics.

    p : integer, optional (default = 2)
        Power parameter for the Minkowski metric. When p = 1, this is
        equivalent to using manhattan_distance (l1), and euclidean_distance
        (l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.

    """
    def __init__(self, n_neighbors = 5, scaler = 'StandardScaler',
                 metric = 'minkowski', p = 2, weights = 'uniform', **kwargs):

        self.n_neighbors = n_neighbors
        self.scaler = scaler
        self.metric = metric
        self.p = p
        self.weights = weights
        self.kwargs = kwargs

    def fit(self, X, y):

        self._y = y.values
        self._fit_X = X.values


    def _predict(self, X):

        # Шкалирование 3 варианта
        scalers_dict = {'StandardScaler':StandardScaler,
                        'MinMaxScaler':MinMaxScaler,
                        'None':None}

        self.scaler_function = scalers_dict[self.scaler]
        if self.scaler_function:
            scaler = self.scaler_function()
            scaler.fit(self._fit_X)
            X_train = scaler.transform(self._fit_X)
            X_test = scaler.transform(X)
        else:
            X_train = self._fit_X
            X_test = X

        metric_dict = {'braycurtis':braycurtis, 'canberra':canberra,
                       'chebyshev':chebyshev, 'cityblock':cityblock,
                       'correlation':correlation, 'cosine':cosine,
                       'dice':dice, 'euclidean':euclidean,
                       'hamming':hamming, 'jaccard':jaccard,
                       'kulsinski':kulsinski, 'mahalanobis':mahalanobis,
                       'matching':matching, 'minkowski':minkowski,
                       'rogerstanimoto':rogerstanimoto, 'russellrao':russellrao,
                       'seuclidean':seuclidean, 'sokalmichener':sokalmichener,
                       'sokalsneath':sokalsneath, 'sqeuclidean':sqeuclidean,
                       'wminkowski':wminkowski, 'yule':yule}

        metric = metric_dict[self.metric]
        distances = cdist(X_test, X_train, metric, self.p, self.weights, **self.kwargs)

        neighbors = np.argsort(distances)[:, :self.n_neighbors]
        probs_1_cl = np.array([np.mean(self._y[neighbor]) for neighbor in neighbors])
        return probs_1_cl

    def predict(self, X):
        """Predict the class labels for the provided data
        Parameters
        ----------
        X : array-like, shape (n_query, n_features), \
                or (n_query, n_indexed) if metric == 'precomputed'
            Test samples.
        Returns
        -------
        y : array of shape [n_samples] or [n_samples, n_outputs]
            Class labels for each data sample.
        """
        classes = self._predict(X).round()
        return classes

    def predict_proba(self, X):
        """Return probability estimates for the test data X.
        Parameters
        ----------
        X : array-like, shape (n_query, n_features), \
                or (n_query, n_indexed) if metric == 'precomputed'
            Test samples.
        Returns
        -------
        p : array of shape = [n_samples, n_classes], or a list of n_outputs
            of such arrays if n_outputs > 1.
            The class probabilities of the input samples. Classes are ordered
            by lexicographic order.
        """
        probs_1_cl = self._predict(X)
        probs_2_cl = np.array(list(zip(1 - probs_1_cl, probs_1_cl)))
        return probs_2_cl

    def score(self, X, y, sample_weight=None):
        """
        Parameters
        ----------
        X : array-like, shape = (n_samples, n_features)
            Test samples.
        y : array-like, shape = (n_samples) or (n_samples, n_outputs)
            True labels for X.
        sample_weight : array-like, shape = [n_samples], optional
            Sample weights.
        Returns
        -------
        score : float
            f1_score of self.predict(X) wrt. y.
        """
        return f1_score(y, self.predict(X), sample_weight=sample_weight)


In [222]:
from sklearn.neighbors import KDTree

In [317]:
tree = KDTree(X, leaf_size=2)          
np.array(tree.data)

array([[ 0.051267 ,  0.69956  ],
       [-0.092742 ,  0.68494  ],
       [-0.21371  ,  0.69225  ],
       [-0.375    ,  0.50219  ],
       [-0.51325  ,  0.46564  ],
       [-0.52477  ,  0.2098   ],
       [-0.39804  ,  0.034357 ],
       [-0.30588  , -0.19225  ],
       [ 0.016705 , -0.40424  ],
       [ 0.13191  , -0.51389  ],
       [ 0.38537  , -0.56506  ],
       [ 0.52938  , -0.5212   ],
       [ 0.63882  , -0.24342  ],
       [ 0.73675  , -0.18494  ],
       [ 0.54666  ,  0.48757  ],
       [ 0.322    ,  0.5826   ],
       [ 0.16647  ,  0.53874  ],
       [-0.046659 ,  0.81652  ],
       [-0.17339  ,  0.69956  ],
       [-0.47869  ,  0.63377  ],
       [-0.60541  ,  0.59722  ],
       [-0.62846  ,  0.33406  ],
       [-0.59389  ,  0.005117 ],
       [-0.42108  , -0.27266  ],
       [-0.11578  , -0.39693  ],
       [ 0.20104  , -0.60161  ],
       [ 0.46601  , -0.53582  ],
       [ 0.67339  , -0.53582  ],
       [-0.13882  ,  0.54605  ],
       [-0.29435  ,  0.77997  ],
       [-0

In [318]:
X.values

array([[ 0.051267 ,  0.69956  ],
       [-0.092742 ,  0.68494  ],
       [-0.21371  ,  0.69225  ],
       [-0.375    ,  0.50219  ],
       [-0.51325  ,  0.46564  ],
       [-0.52477  ,  0.2098   ],
       [-0.39804  ,  0.034357 ],
       [-0.30588  , -0.19225  ],
       [ 0.016705 , -0.40424  ],
       [ 0.13191  , -0.51389  ],
       [ 0.38537  , -0.56506  ],
       [ 0.52938  , -0.5212   ],
       [ 0.63882  , -0.24342  ],
       [ 0.73675  , -0.18494  ],
       [ 0.54666  ,  0.48757  ],
       [ 0.322    ,  0.5826   ],
       [ 0.16647  ,  0.53874  ],
       [-0.046659 ,  0.81652  ],
       [-0.17339  ,  0.69956  ],
       [-0.47869  ,  0.63377  ],
       [-0.60541  ,  0.59722  ],
       [-0.62846  ,  0.33406  ],
       [-0.59389  ,  0.005117 ],
       [-0.42108  , -0.27266  ],
       [-0.11578  , -0.39693  ],
       [ 0.20104  , -0.60161  ],
       [ 0.46601  , -0.53582  ],
       [ 0.67339  , -0.53582  ],
       [-0.13882  ,  0.54605  ],
       [-0.29435  ,  0.77997  ],
       [-0

In [320]:
[tree.query([el], k=10) for el in X.values[0:3]]

[(array([[ 0.        ,  0.09567374,  0.14474922,  0.15135558,  0.15254227,
           0.19060457,  0.19782518,  0.2052146 ,  0.21937566,  0.224657  ]]),
  array([[ 0, 92,  1, 50, 17, 59, 16, 52, 93, 18]])),
 (array([[ 0.        ,  0.08196246,  0.08854379,  0.12118867,  0.13158   ,
           0.13586141,  0.13941642,  0.14474922,  0.14633391,  0.21768947]]),
  array([[ 1, 18, 32,  2, 52, 31, 17,  0, 28, 92]])),
 (array([[ 0.        ,  0.04097729,  0.05957686,  0.11915372,  0.12118867,
           0.12128688,  0.16426488,  0.18418385,  0.19065139,  0.20820439]]),
  array([[ 2, 18, 32, 29,  1, 31, 28, 52, 88, 17]]))]

In [225]:
dist, ind = tree.query([X[0]], k=3)                
print(ind)  # indices of 3 closest neighbors
print(dist)

[[0 3 1]]
[[ 0.          0.19662693  0.29473397]]
